#### Download data to Master node HDFS

In [ ]:
%%bash
wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-05.csv;

#### Move data to gs

In [ ]:
%%bash
gsutil cp yellow_tripdata_2019-05.csv gs://bucket-dataproc/landing;

#### imports

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType,StructField, StringType, LongType
from pyspark.sql.functions import current_timestamp, current_date

In [ ]:
spark = (SparkSession
        .builder
        .config("spark.memory.offHeap.enabled","true")
        .config("spark.memory.offHeap.size","100mb")
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .getOrCreate())

In [ ]:
df = spark.read.csv("gs://bucket-dataproc/landing",header=True, inferSchema=True)

In [ ]:
df = df.withColumn("processed_at", current_timestamp())
df = df.withColumn("load_date", current_date())

#### saveto GS

In [ ]:
df.write.save("gs:bucket-dataproc/processed/p_yellow_tripdata_2019-05.csv", format='csv', header=True')